In [0]:
%run "../../../config/setup"


Get customers_bronze table and deduplicate customer_id

In [0]:
CREATE OR REPLACE TABLE customers_dedupped AS
  SELECT customer_id, max(first_name) AS first_name, max(last_name) AS last_name, max(email) AS email, max(phone) AS phone, max(city) AS city, max(registration_date) AS registration_date FROM customers_bronze
  WHERE customer_id IS NOT NULL 
  GROUP BY customer_id
  

In [0]:
GRANT SELECT ON TABLE customers_dedupped TO devs

In [0]:
-- will be commented out for optimization of pipeline
-- SELECT max(row_count) <= 1 AS valid FROM 
--   (SELECT customer_id, count(*) AS row_count FROM customers_dedupped
--   WHERE customer_id IS NOT NULL
--   GROUP BY customer_id)

In [0]:
CREATE OR REPLACE TABLE customers_silver AS
  SELECT
    customer_id,
    first_name,
    last_name,
    phone,
    city,
    email,
    regexp_extract(email, '^(.*)@(.*)', 1) as email_local,
    regexp_extract(email, '@(.*)', 1) as email_domain,
    CAST(registration_date AS DATE) AS registration_date
  FROM customers_dedupped

In [0]:
GRANT SELECT ON TABLE customers_silver TO devs;